In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Lecture 16 (optional) - Analytics with SQLite and Pandas

# Table of Contents
* [Lecture 16 - Analytics with SQLite](#Lecture-16---Analytics-with-SQLite)
	* &nbsp;
		* [Learning Outcomes](#Learning-Outcomes)
		* [Create and populate SQLite tables](#Create-and-populate-SQLite-tables)
        * [Subqueries](#Subqueries)
        * [Creating Bins or Value bands](#Creating-Bins-or-Value-bands)
        * [RFM with SQLite](#RFM-with-SQLite)
        * [Classifying our customers using CASE statements](#Classifying-our-customers-using-CASE-statements)
        * [Final exercise - Visits to store, target product](#Final-exercise---Visits-to-store,-target-product)
        

In this lecture we are going to practice creating and populating tables with SQLite.  We will look a bit more a subqueries and binning data.  We are then going to look at some basic analytics with SQLite and Pandas - namely binning customer data for the purposes of an RFM analysis.  Finally, there is a loyalty customer analysis exercise.

## Learning Outcomes

At the end of this lecture, you should be able to:

* Be more confident with defining a database schema with SQLite
* Be more confident with iteratively populating database tables with SQLite
* Use subqueries
* Bin data with SQLite and visualise that data with python
* Experiment with binning for the purposes of customer segmentation in an RFM analysis
* Try a loyalty customer analysis


In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
connection = sqlite3.connect('../datasets/mySQLiteDB.sl3')
cursor = connection.cursor()

## Create and populate SQLite tables

In this notebook we will be using the product, line, invoice, vendor and customer tables from the first half of semester SQL content.  Csv versions of these are available on Stream in the datasets folder.  The invoice table has had dates altered to suit our analysis.

**Exercise:** Read in "product.csv", "line.csv", "vendor.csv", "invoice.csv" and "customer.csv" into separate dataframes and check they have been imported correctly

Now we will build a schema for these tables (i.e. define all of the tables with constraints)

In [ ]:
cursor.execute('DROP TABLE IF EXISTS PRODUCT')

table = """
            CREATE TABLE PRODUCT (
            P_CODE VARCHAR2(10) CONSTRAINT PRODUCT_P_CODE_PK PRIMARY KEY,
            P_DESCRIPT VARCHAR2(35) NOT NULL,
            P_INDATE DATE NOT NULL,
            P_QOH NUMBER NOT NULL,
            P_MIN NUMBER NOT NULL,
            P_PRICE NUMBER(8,2) NOT NULL,
            P_DISCOUNT NUMBER(5,2) NOT NULL,
            V_CODE NUMBER);
            """
cursor.execute(table)

In [ ]:
cursor.execute('DROP TABLE IF EXISTS CUSTOMER')
table = """
            CREATE TABLE CUSTOMER (
            CUS_CODE NUMBER PRIMARY KEY,
            CUS_LNAME VARCHAR(15) NOT NULL,
            CUS_FNAME VARCHAR(15) NOT NULL,
            CUS_INITIAL CHAR(1),
            CUS_AREACODE CHAR(3) DEFAULT '615' NOT NULL CHECK(CUS_AREACODE IN ('615','713','931')),
            CUS_PHONE CHAR(8) NOT NULL,
            CUS_BALANCE NUMBER(9,2) DEFAULT 0.00,
            CONSTRAINT CUS_UI1 UNIQUE(CUS_LNAME,CUS_FNAME)
            );
                """

cursor.execute(table)

In [ ]:
cursor.execute('DROP TABLE IF EXISTS LINE')
table = """
        CREATE TABLE LINE (
            INV_NUMBER NUMBER NOT NULL,
            LINE_NUMBER NUMBER(2,0) NOT NULL,
            P_CODE VARCHAR(10) NOT NULL,
            LINE_UNITS NUMBER(9,2) DEFAULT 0.00 NOT NULL,
            LINE_PRICE NUMBER(9,2) DEFAULT 0.00 NOT NULL,
            PRIMARY KEY (INV_NUMBER,LINE_NUMBER),
            CONSTRAINT LINE_UI1 UNIQUE(INV_NUMBER, P_CODE)
            );
            """
cursor.execute(table)

In [ ]:
cursor.execute('DROP TABLE IF EXISTS VENDOR')
table = """
        CREATE TABLE VENDOR ( 
        V_CODE INTEGER, 
        V_NAME VARCHAR(35) NOT NULL, 
        V_CONTACT VARCHAR(15) NOT NULL, 
        V_AREACODE CHAR(3) NOT NULL, 
        V_PHONE CHAR(8) NOT NULL, 
        V_STATE CHAR(2) NOT NULL, 
        V_ORDER CHAR(1) NOT NULL, 
        PRIMARY KEY (V_CODE)
);
            """
cursor.execute(table)

**Exercise:** Add the invoice table to the schema and commit() your changes to the database

Now we are going to populate the tables iteratively using the substitution method from lecture 14.  First, we define row insertion statements for each table that as many placeholders as there are columns in the dataframe

In [ ]:
add_row_product = """
                            INSERT INTO PRODUCT 
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                        """
add_row_customer = """
                            INSERT INTO CUSTOMER 
                            VALUES (?, ?, ?, ?, ?, ?, ?)
                        """
add_row_line = """
                            INSERT INTO LINE 
                            VALUES (?, ?, ?, ?, ?)
                        """
add_row_vendor = """
                            INSERT INTO VENDOR 
                            VALUES (?, ?, ?, ?, ?, ?, ?)
                        """


**Exercise:** create an insertion statement for the invoice table

We can iteratively populate our tables with a for loop, as we did in lecture 14.

**Product**

In [ ]:
for index, row in product.iterrows():
    substitution_values = (row['P_CODE'], row['P_DESCRIPT'], row['P_INDATE'],\
                           row['P_QOH'], row['P_MIN'], row['P_PRICE'], \
                           row['P_DISCOUNT'], row['V_CODE'])
    cursor.execute(add_row_product, substitution_values)

connection.commit()

In [ ]:
#Check
sql_statement = 'SELECT * FROM PRODUCT;'
cursor.execute(sql_statement)


for (P_CODE, P_DESCRIPT, P_INDATE, P_QOH, P_MIN, P_PRICE, P_DISCOUNT,V_CODE) in cursor:
    print(P_CODE, P_DESCRIPT, P_INDATE, P_QOH, P_MIN, P_PRICE, P_DISCOUNT,V_CODE)

**Customer**

In [ ]:
for index, row in customer.iterrows():
    substitution_values = (row['CUS_CODE'], row['CUS_LNAME'], row['CUS_FNAME'],\
                           row['CUS_INITIAL'], row['CUS_AREACODE'], row['CUS_PHONE'], \
                           row['CUS_BALANCE'])
    cursor.execute(add_row_customer, substitution_values)

connection.commit()

In [ ]:
#Check
sql_statement = 'SELECT * FROM CUSTOMER;'
cursor.execute(sql_statement)


for (CUS_CODE, CUS_LNAME, CUS_FNAME, CUS_INITIAL, CUS_AREACODE, CUS_PHONE, CUS_BALANCE) in cursor:
    print(CUS_CODE, CUS_LNAME, CUS_FNAME, CUS_INITIAL, CUS_AREACODE, CUS_PHONE, CUS_BALANCE)

**Line**

In [ ]:
for index, row in line.iterrows():
    substitution_values = (row['INV_NUMBER'], row['LINE_NUMBER'], row['P_CODE'],\
                           row['LINE_UNITS'], row['LINE_PRICE'])
    cursor.execute(add_row_line, substitution_values)

connection.commit()

In [ ]:
#Check
sql_statement = 'SELECT * FROM LINE;'
cursor.execute(sql_statement)


for (INV_NUMBER, LINE_NUMBER, P_CODE, LINE_UNITS, LINE_PRICE) in cursor:
    print(INV_NUMBER, LINE_NUMBER, P_CODE, LINE_UNITS, LINE_PRICE)

**Vendor**

In [ ]:
for index, row in vendor.iterrows():
    substitution_values = (row['V_CODE'], row['V_NAME'], row['V_CONTACT'],\
                           row['V_AREACODE'], row['V_PHONE'], row['V_STATE'], \
                           row['V_ORDER'])
    cursor.execute(add_row_vendor, substitution_values)

connection.commit()

In [ ]:
#Check
sql_statement = 'SELECT * FROM VENDOR;'
cursor.execute(sql_statement)


for (V_CODE, V_NAME, V_CONTACT, V_AREACODE, V_PHONE, V_STATE, V_ORDER) in cursor:
    print(V_CODE, V_NAME, V_CONTACT, V_AREACODE, V_PHONE, V_STATE, V_ORDER)

**Exercise:** Insert the invoice rows into the invoice table and check the result.

## Subqueries
A closer look at subqueries, using the pandas SQL methods.

A subquery is a query embedded or nested inside another query.  The inner query will be executed first and the outer query will work with the result of the subquery.  

Subqueries are wrapped up in parentheses.

### With a projection clause (i.e. making a column with a subquery)

We can use a subquery where we would put a column name.  You should give each subquery an alias to improve the readability of your query and also of the result set. 

Take the following query:


In [ ]:
sql_statement = """
                SELECT AVG(P_PRICE) FROM PRODUCT
                """
pd.read_sql_query(sql_statement, connection)

We can use this as a subquery within the column list (projection) of another query:

In [ ]:
sql_statement = """
                SELECT 
                P_CODE, 
                P_PRICE, 
                (SELECT AVG(P_PRICE) FROM PRODUCT) AS "AVERAGE PRICE", 
                P_PRICE - (SELECT AVG(P_PRICE) FROM PRODUCT) AS DIFFERENCE
                FROM PRODUCT;
                """
pd.read_sql_query(sql_statement, connection)


**Exercise:** Alter the above query so that the difference between the product price and maximum product price is found.

### With WHERE
- Uses a SELECT subquery on the right side of a WHERE comparison expression
- Value generated by the subquery must be of a comparable data type
- Can be used in combination with joins

Eg. to return a list of vendors who have not supplied products:

First, let's look at a subquery that finds all vendor codes in the product table (this is a list of vendors that *have* supplied products):


In [ ]:
sql_statement = """
                SELECT DISTINCT V_CODE FROM PRODUCT WHERE V_CODE IS NOT NULL;

                """
pd.read_sql_query(sql_statement, connection)

To find return a list of vendors who have not supplied products, we would look for vendors in the vendor table who are not in the above list:

In [ ]:
sql_statement = """
                SELECT 
                V_CODE, 
                V_NAME 
                FROM VENDOR
                WHERE V_CODE NOT IN 
                (SELECT DISTINCT V_CODE FROM PRODUCT WHERE V_CODE IS NOT NULL);

                """
pd.read_sql_query(sql_statement, connection)

**Exercise:** Alter the above query so that we select from the vendor table only those vendors who have supplied mroe than one product.  A GROUP BY and HAVING clause will be needed in the subquery

### With HAVING
- HAVING clause restricts the output of a GROUP BY query by applying conditional criteria to the grouped rows

Eg., to list all products with total quantity sold greater than average quantity sold:

First, a look at the subquery we will need:


In [ ]:
sql_statement = """
                SELECT AVG(LINE_UNITS) FROM LINE;

                """
pd.read_sql_query(sql_statement, connection)

We can use this in our HAVING clause:

In [ ]:
sql_statement = """
                SELECT P_CODE, SUM(LINE_UNITS) TOTAL_UNITS FROM LINE
                GROUP BY P_CODE
                HAVING SUM(LINE_UNITS) > (SELECT AVG(LINE_UNITS) FROM LINE);

                """
pd.read_sql_query(sql_statement, connection)

Note that none of the products returned in the result set have sold less than three units

**Exercise:** Alter the above query to select products that have in total sold less than the maximum number of items sold in any line.

### With FROM

The FROM clause is where we specify the table that we are querying, e.g. to query the product table, we could use this query:

> SELECT * FROM PRODUCT

We can use a subquery with FROM rather than specify a table name.  Queries return result sets which in turn are tables themselves.  By using a subquery with the FROM clause, it is like creating a table on the fly from which we make our query.  

The query below sums up customer spend to get a total spend for each customer:

In [ ]:
sql_statement = """
                SELECT 
                I.CUS_CODE, 
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS Total_Spend
                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY 
                I.CUS_CODE

                """
pd.read_sql_query(sql_statement, connection)

We could now query this as if it were an existing table by including it as a subquery after the FROM clause.  Note that we give the subquery a table alias that we refer to when selecting data from it (SUBQ in this case).  Here we select the average total spend from the table above:

In [ ]:
sql_statement = """
                SELECT
                AVG(SUBQ.Total_Spend) as "Average Total Customer Spend"
                FROM


                (SELECT 
                I.CUS_CODE, 
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS Total_Spend
                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY 
                I.CUS_CODE) SUBQ 
;

"""
pd.read_sql_query(sql_statement, connection)



**Exercise:** Create a query that calculates the average total units sold across all products.  

In the FROM subquery you will sum up the number of units sold for each product using the line table.  The outer query will then calculate the average total units sold across all products.

## Creating Bins or Value bands 
Often we want to bin customers or products up into value bands to get an idea of how many sit within each band.  For instance, we might want to know how many products cost up to \\$10, how many cost \\$10-\\$20, how many cost \\$20-\\$30 and so on.

In Oracle SQL we might use CEIL or FLOOR to achieve this.  e.g. to show a $10 price band for products we could use this query:

>SELECT FLOOR(P_PRICE/10)*10 FROM PRODUCT;

For $50 price bands, we could use this query:

>SELECT FLOOR(P_PRICE/50)*50 FROM PRODUCT;

In SQLLite there is no CEIL or FLOOR, so we have to find another way.  

The following formulae will work, but only when the value is greater than zero. 
    - The lower limit of the bin can be found with ROUND((VALUE - 0.5*<bin_width>)/<bin_width>, 0)*<bin_width> 
    - The upper limit of the bin can be found with ROUND((VALUE - 0.5*<bin_width>)/<bin_width>, 0)*<bin_width>
   
   
So if we had bins of size 10, the calculations would be:

 - Lower = ROUND((VALUE - 5)/10, 0)*10
 - Upper = ROUND((VALUE + 5)/10, 0)*10

And for bins of size 50, the calculations would be:

 - Lower = ROUND((VALUE - 25)/50, 0)*50
 - Upper = ROUND((VALUE + 25)/50, 0)*50

For instance, let’s look at what price band each product in the product table sits in, in $10 increments:

In [ ]:
sql_statement = """

                SELECT 
                P_CODE, 
                P_PRICE,
                ROUND((P_PRICE-5)/10,0)*10 as "LOWER PRICE BAND",
                ROUND((P_PRICE+5)/10,0)*10 as "UPPER PRICE BAND"
                FROM PRODUCT
                ;
"""

pd.read_sql_query(sql_statement, connection)


### Grouping product values into price bins

Putting this together with GROUP BY, we get a count of products in each price band.  The following query only calculates the lower price band:

In [ ]:
#Note that in Oracle SQL a group by cannot refer to an alias
# it would need to be written as: GROUP BY ROUND((P_PRICE-0.5)/10, 0)*10
# or even better GROUP BY FLOOR(P_PRICE/10)*10 (because we can use "FLOOR" in Oracle)

sql_statement = """

                SELECT 
                ROUND((P_PRICE-5)/10,0)*10 as "LOWER PRICE BAND",
                COUNT(P_CODE) AS "PRODUCT COUNT"
                FROM PRODUCT
                GROUP BY 
                "LOWER PRICE BAND"
                ORDER BY "LOWER PRICE BAND";
"""

product_bands = pd.read_sql_query(sql_statement, connection)
product_bands

We could quickly plot the result, for EDA purposes

In [ ]:
product_bands.set_index('LOWER PRICE BAND', inplace = True)
product_bands.plot(kind = "bar");

This result is a little misleading as the x-axis does not reflect the full range of possible price bands.  We can use matplotlib methods to address this, primarily, the xticks method which will define the span of our plot.  (You may wonder how we know this stuff - we have spent a lot of time reading the documentation to solve various problems.)

In [ ]:
plt.figure(figsize = (15, 6))
plt.xticks(np.arange(0,280, step = 10));

Putting this together with the data..

In [ ]:
plt.figure(figsize = (15, 6))
plt.xticks(np.arange(0,280, step = 10))
plt.bar(product_bands.index, product_bands['PRODUCT COUNT'], \
        color = "#7d16ea", width = 10, align = 'edge' )
plt.grid(which='major', axis='y', linestyle='dashed', linewidth=0.7);
plt.title("Product counts by price band")
plt.xlabel('Price band ($)')
plt.ylabel('Product count'); 
#the semi-colon will suppress the output of this final command.  Try removing it to see what happens

**Exercise:** Group products into price bands of $20, and plot the result

### Grouping customer spend into bins

More commonly, in the course of creating a single customer view, we might calculate each customer’s total spend.  We could then group up customers into spend bands. 

Let’s first look at how we find customer spend.  We need to link the customers in the invoice table to the spend in the line table.  The path is:

Invoice------------------------------->Line

Using:

- I.INV_NUMBER = L. INV_NUMBER

In [ ]:
#The spend on each line is found by multiplying the price by the number of units bought
sql_statement = """
                SELECT
                I.CUS_CODE,
                I.INV_NUMBER, 
                L.LINE_UNITS*L.LINE_PRICE AS Spend_Per_Line
                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                ;
                """
pd.read_sql_query(sql_statement, connection)

Now we have the amount each customer spent on each line in an invoice, we can add it up to get total spend by customer...here comes the GROUP BY:

In [ ]:
sql_statement = """
                SELECT 
                I.CUS_CODE, 
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS Total_Spend
                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY 
                I.CUS_CODE
                ;
                """
tot_cust_spend = pd.read_sql_query(sql_statement, connection)
tot_cust_spend

We could take this further, and group total spend into spend bands and get the number of customers in each band.

We will group by bands of $250 in this example.  

We could either create a temporary table from the result in the previous slide and query that, or we could turn our previous query into a subquery as below...


In [ ]:
sql_statement = """
                SELECT
                ROUND((SUBQ.Total_Spend-125)/250,0)*250 as "Lower Spend Band",
                ROUND((SUBQ.Total_Spend+125)/250,0)*250 as "Upper Spend Band",
                COUNT(SUBQ.CUS_CODE) as "Customer Count"
                FROM


                (SELECT 
                I.CUS_CODE, 
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS Total_Spend
                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY 
                I.CUS_CODE) SUBQ 

                GROUP BY 
                "Lower Spend Band",
                "Upper Spend Band"
                ORDER BY "Lower Spend Band";

"""
cust_bands = pd.read_sql_query(sql_statement, connection)
cust_bands

There is a lot going on in that query, let's break it down

- First, note that because the outer query is selecting from the result set produced by our subquery, we need to refer to the subquery columns using the subquery alias:


                SELECT
                ROUND((SUBQ.Total_Spend-125)/250,0)*250 as "Lower Spend Band",
                ROUND((SUBQ.Total_Spend+125)/250,0)*250 as "Upper Spend Band",
                COUNT(SUBQ.CUS_CODE) as "Customer Count"
                FROM

- This is the subquery - it is the same query we ran just before. It has been given the alias "SUBQ":

                (SELECT 
                I.CUS_CODE, 
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS Total_Spend
                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY 
                I.CUS_CODE) SUBQ 

- The final group by is a grouping of the result of the outer query so it refers to the outer query columns.
- We have to group on both bands as they are not being aggregated.  

                GROUP BY 
                "Lower Spend Band",
                "Upper Spend Band"
                ORDER BY "Lower Spend Band";

We can plot our customers according to how many are in each spend bin.  The plot is not the most interesting one.  Usually we would get a sense of the distribution of customers and make changes to the size of our bins according to what we see

In [ ]:
cust_bands[['Lower Spend Band', 'Customer Count']].set_index('Lower Spend Band')\
.plot(kind = "bar", align = 'edge');

The product plot gives a better idea of how we might use such a plot to choose groupings based on value bands - there are possibly three groups here - Up to \\$50, \\$50-\\$120, and \\$120+

In [ ]:
plt.figure(figsize = (15, 6))
plt.xticks(np.arange(0,280, step = 10))
plt.bar(product_bands.index, product_bands['PRODUCT COUNT'], \
        color = "#7d16ea", width = 10, align = 'edge' )
plt.grid(which='major', axis='y', linestyle='dashed', linewidth=0.7);
plt.title("Product counts by price band")
plt.xlabel('Price band ($)')
plt.ylabel('Product count'); 

### Using Pandas to look at distributions
Note that to visualise a distribution with python/pandas, we can simply use a histogram to similar effect. 



In [ ]:
# A histogram for product prices
product.P_PRICE.hist(bins = 20);

In [ ]:
# histogram for total customer spend
tot_cust_spend.Total_Spend.hist();

You can choose the number of bins

In [ ]:
tot_cust_spend.Total_Spend.hist(bins = 2);

Alternatively, you can define bin sizes

In [ ]:
plt.hist(tot_cust_spend.Total_Spend, bins=[0, 100, 200, 300, 400, 500]);

## RFM with SQLite
RFM analysis is a way of segmenting customers based on their purchasing habits.  Customers are classified according to time since last transaction (Recency), how many times they have transacted in a given period - e.g. in the last 12 months - (Frequency), and what their total spend was over that period (Monetary).

Note that before we classify customers:

- we need to know what our customers raw RFM values are (i.e. what is their total spend, recency and frequency)
- we need to figure out what bin sizes we will use (which is  we did before) 

At which point we can classify our customers into the bins depending on the raw RFM values.

We have already figured out good bin sizes for Monetary above (total customer spend) using a subquery.  

This time we are going to makes things a bit simpler.  We are going to create a CUSTOMER_RFM table that has all of the raw aggregates we need in it.  We can then directly query that table.



### Creating a Customer RFM Table
We are going to create a view of our customers that includes their raw recency, frequency and monetary values. We will use those values to classify our customers a bit later.

First, let's look how we get to the Recency, Frequency and Monetary values one by one before we bring it all into one table.


### Recency
This is the number of days since the last transaction for each customer.  

I am assuming that we are interested in the most recent purchase with reference to the end of the financial year.

Note that to find days between two dates in SQLite, we need to convert them to julian dates.  In Oracle, we would use the TO_DATE function to convert the string '2019-03-31' to a date and simply subtract one date from the other.

In [ ]:
sql_statement = """
                SELECT 
                CUS_CODE, 
                julianday('2019-03-31')-MAX(julianday(INV_DATE)) AS Recency
                FROM INVOICE
                GROUP BY CUS_CODE;
                """
pd.read_sql_query(sql_statement, connection)



### Frequency
Let's look at the frequency for each customer.

All we need to know is in the invoice table (invoice number and customer code).  We simply need to count the number of invoices a customer has generated.

In [ ]:
sql_statement = """
                SELECT 
                
                CUS_CODE,
                COUNT(INV_NUMBER) AS Frequency
                FROM INVOICE 
                GROUP BY CUS_CODE
                ;
                """
pd.read_sql_query(sql_statement, connection)


### Monetary
We already worked out the monetary calculation (total customer spend) in the previous section:

In [ ]:
sql_statement = """
                SELECT 
                I.CUS_CODE, 
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS Monetary
                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY 
                I.CUS_CODE
                ;
                """
pd.read_sql_query(sql_statement, connection)


### Bringing it altogether into an customer RFM table

We can bring all of the above queries together to create a single customer RFM table containing all of these values, as below:

In [ ]:
connection.execute('DROP TABLE IF EXISTS CUSTOMER_RFM')

sql_statement = """
                CREATE TABLE CUSTOMER_RFM
                AS
                SELECT 
                I.CUS_CODE, 
                
                julianday('2019-03-31')-MAX(julianday(I.INV_DATE)) AS Recency,
                COUNT(DISTINCT I.INV_NUMBER) AS Frequency,
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS "Monetary"

                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY I.CUS_CODE;
                """
connection.execute(sql_statement)
connection.commit()


That statement needs some explaining:

- First, note that we can create a table from the result set of a query:


               CREATE TABLE CUSTOMER_RFM
               AS
               SELECT 
               I.CUS_CODE,

- Second, note the COUNT(DISTINCT...) here.  We need to be careful – our join will return some invoice numbers multiple times as there will be a row per invoice line item, so we need to be sure to count distinct invoice numbers

- Third, note that these are the aggregates that we will use for our classification.

                julianday('2019-03-31')-MAX(julianday(I.INV_DATE)) AS Recency,
                COUNT(DISTINCT I.INV_NUMBER) AS Frequency,
                SUM(L.LINE_UNITS*L.LINE_PRICE) AS "Monetary"

- Fourth, note we are joining invoice to line

                FROM INVOICE I JOIN LINE L
                ON(I.INV_NUMBER = L.INV_NUMBER)
                GROUP BY I.CUS_CODE;

**Exercise:** Display the CUSTOMER_RFM table

### Choosing bin sizes - Recency

Now we need to decide on bins for our data.  In reality we could eyeball this data and decide, but let's assume it is a large dataset and we cannot do that.  First we are going to create bins using SQL - trying out bins of size ten.  

In [ ]:
sql_statement = """
            SELECT 
            ROUND((Recency-5)/10,0)*10 AS "Recency Band Lower",
            ROUND((Recency+5)/10,0)*10 AS "Recency Band Upper",
            COUNT(CUS_CODE) AS "Customer Count"
            FROM
            CUSTOMER_RFM
            
            GROUP BY 
            "Recency Band Lower", 
            "Recency Band Upper"
            ORDER BY "Recency Band Lower";

"""
recency_binned = pd.read_sql_query(sql_statement, connection)
recency_binned

**Exercise:** Plot the recency_binned dataframe created above.  For comparison, do the same with a pandas histogram using the un-binned recency data (the Recency column in the CUSTOMER_RFM table). 

### Recency bin size decision
Let's stick with bins of size 10 for Recency.

### Choosing bin sizes - Frequency

You won't need to calculate a ceiling or floor for Frequency as you will be grouping by count.  In FMCG analyses it would make sense to create bands for frequency as customers tend to make many visits, but this is not the case in this tiny dataset.

In [ ]:
sql_statement = """
                SELECT
                Frequency,
                COUNT(CUS_CODE) 
                AS "Customer count"
                FROM
                CUSTOMER_RFM
                
                GROUP BY 
                Frequency
                ORDER BY Frequency
                ;
                """
frequency_binned = pd.read_sql_query(sql_statement, connection)
frequency_binned


**Exercise:** Plot the binned frequency result set you just created with SQLite.  For comparison, do the same with a pandas histogram using the un-binned frequency data.

### Frequency bin size decision
Our "bin sizes" will be of size one in this case.

### Choosing bin sizes - Monetary 

We have already done this, but let's do it again...

In [ ]:
sql_statement = """
                SELECT ROUND((Monetary-125)/250,0)*250 AS "Monetary Band Lower",
                ROUND((Monetary+125)/250,0)*250 AS "Monetary Band Upper",
                COUNT(CUS_CODE) AS "Customer Count"
                FROM
                CUSTOMER_RFM
                
                GROUP BY 
                "Monetary Band Lower"
                ORDER BY "Monetary Band Lower", "Monetary Band Upper"
                ;
                """
monetary_binned = pd.read_sql_query(sql_statement, connection)
monetary_binned


### Monetary bin size decision
We will stick with bins of size $250.  

With a real-world dataset, we would be experimenting with different bin sizes for all three categories.

### RFM crosstab
Once we have experimented with different bands and looked at the distribution of customer counts within them, we can build an RFM table according to our chosen cut-offs.  We can then classify our customers and populate the RFM crosstab with counts (or percentages) and make decisions around how we would like to group these customers according to behaviours.  Below is a very basic attempt at this which only uses one of the dimensions (Recency):


![](../figures/RFM.PNG)


## Classifying our customers using CASE statements

We have the raw RFM aggregates for our customers, and we have decided on bin sizes.  Now we can classify our customers into bins.  Here we will be usign CASE statements.

With PL/SQL you have access to functionality that you get with procedural programming languages, namely control flow statements, user defined functions, variables and loops.  Additionally with PL/SQL we can create stored procedures.

SQLite does not directly support PL/SQL, however CASE statements can be used.  A case statement is a way of writing a control flow (if/else) statement without having to call a procedure. Case statements are also used in Oracle SQL

The basic syntax is:

   >WHEN *CONDITION 1* THEN *VALUE 1*<br/>
   >WHEN *CONDITION 2* THEN *VALUE 2*<br/>
   >WHEN *CONDITION 3* THEN *VALUE 3*<br/>
   >...<br/>
   >ELSE *VALUE N*<br/>
   >END

Case statements are often used with a select statement to define column values, they can also be used to update column values.

   


### CASE Statements with SELECT
Example with SELECT – Classify customers according to frequency


In [ ]:
sql_statement = """
        SELECT 
        CUS_CODE, 
        COUNT(INV_NUMBER) AS Frequency,
        CASE 
            WHEN COUNT(INV_NUMBER) = 1 THEN 'Not frequent'
            WHEN COUNT(INV_NUMBER) = 2 THEN 'Kind of frequent'
            ELSE 'Best customer'
        END Frequency_class
        FROM
        INVOICE
        GROUP BY CUS_CODE
        ;
        """
pd.read_sql_query(sql_statement, connection)

### CASE Statements with UPDATE
We can also use a case statement to update a table.  We are going to add classifications to the CUSTOMER_RFM table we created earlier:

In [ ]:
pd.read_sql_query('select * from CUSTOMER_RFM', connection)

First, we add columns to our table ready for our classifications (these are the columns we will update with case statements):

In [ ]:

sql_statement = 'ALTER TABLE CUSTOMER_RFM ADD COLUMN Recency_Class NUMBER(3)'
cursor.execute(sql_statement)
sql_statement = 'ALTER TABLE CUSTOMER_RFM ADD COLUMN Frequency_Class VARCHAR(2)'
cursor.execute(sql_statement)
sql_statement = 'ALTER TABLE CUSTOMER_RFM ADD COLUMN Monetary_Class VARCHAR(20)'
cursor.execute(sql_statement)


#commit the changes
connection.commit()

In [ ]:
pd.read_sql_query('select * from CUSTOMER_RFM', connection)

And now for an update using a case statement.  We will update recency first:

In [ ]:
sql_statement = """
        UPDATE CUSTOMER_RFM
        SET Recency_Class = 
            CASE 
                WHEN Recency < 10 THEN 'Up to 10'
                WHEN Recency < 20 THEN '10-20'
                WHEN Recency < 30 THEN '20-30'
                ELSE '30+'
            END;
    """
connection.execute(sql_statement)
connection.commit()

In [ ]:
pd.read_sql_query('select * from CUSTOMER_RFM', connection)

**Exercise:** Write and run two more updates so that your Frequency_Class and Monetary_Class columns look like this:

| CUS_CODE | ... |Frequency_Class | Monetary_Class | 
| --- | --- | --- | --- |
|10011| ... | 3 | 250 - 500 | 
|10012| ... | 1 | Up to 250 | 
|10014| ... | 2 | 250 - 500 | 
|10015| ... | 1 | Up to 250 | 
|10018| ... | 1 | Up to 250 | 



**Exercise:** This one may take a while...

Save your CUSTOMER_RFM table into a pandas dataframe object.  Use pandas grouping or pivoting or crosstab with some reshaping techniques (and probably a fillna(0) somewhere along the way) to create an RFM table like this:


| . | Monetary ($) |250-500 | Up to 250 | 
| --- | --- | --- | --- |
| **Recency(days)** | **Frequency** |. | .| 
|20-30| 1 | 0 | 1 |
|.| 2 | 0 | 0 |
|.| 3 | 0 | 0 |
|10-20| 1 | 0 | 1 |
|.| 2 | 1 | 0 |
|.| 3 | 0 | 0 |
|Up to 10| 1 | 0 | 1 |
|.| 2 | 0 | 0 | 
|.| 3 | 1 | 0 |


An analysis of a larger dataset might have yielded this result (converted to percentages)

| . | Monetary ($) | Up to 250 | 250-500 | 500+ |
| --- | --- | --- | --- | --- |
| **Recency(days)** | **Frequency** | . | . | . | 
|6 months +| 1-5 | 10\% | 4\% | 1\%|
|. | 6-10 | 4\% | 6\% | 3\% |
|. | 11+ | 1\% | 2\% | 1\% |
|2-6 months | 1-5 | 1\% | 2\% | 3\% |
|. | 6-10 | 5\% | 7\% | 5\% |
|. | 11+ | 2\% | 1\% | 7\%|
|Up to 2 months | 1-5 | 11\% | 1\% | 9\% |
|. | 6-10 | 1\% | 1\% | 4\% |
|. | 11+ | 0\% | 3\% | 5\% |


At this point in the analysis you would use this table to help define customer segments, and you would give those segments names.  This is out of the course scope – but you are welcome to try!


## Final exercise - Visits to store, target product

For this exercise, you will need to load in basket_table.csv and transaction_table.csv which you can download from the datasets folder.

Try the exercise with SQLite, and then replicate with pandas

- A department store has aggressively promoted a target product and would like to compare the spend of customers in their loyalty program who purchased that product vs those in their loyalty program who did not.  (note that they are limited to investigating loyalty customers as non-loyalty customers cannot be uniquely identified).
- The target product code is 'AX34'
- They are interested in spend-per-visit, not spend-per-transaction.  A department store has many counters, and in one visit a customer may visit more than one counter and make a transaction.  A single visit is defined as transaction/s made on the same day by the same person. 
- Your task is to:
    - Create and populate basket and transaction SQL tables with the csvs given
    - Inspect the tables to understand how they relate
    - Create a visits table that looks like this (including a flag identifying whether the target product was bought in a visit):

| CUS_CODE | TRANS_DATE | VISIT_TOTAL | BOUGHT_TARGET | 
| --- | --- | --- | --- |
104 | 04/02/2019 | 374.9 | 1 |
100 | 01/02/2019 | 146.9 | 1 | 
102 | 05/02/2019 | 2.3 | 0 |
100 | 04/02/2019 | 234.6 | 1 |
103 | 03/02/2019 | 10.8 | 0 |
101 | 05/02/2019 | 401.8 | 1 |
101 | 02/02/2019 | 357.3 | 1 |
101 | 01/02/2019 | 56.7 | 0 |
100 | 03/02/2019 | 135.2 | 1 |
104 | 05/02/2019 | 215.6 | 0 |


- Work out the average spend per visit where the target product was purchased vs when it was not

| Ave visit spend target |  
| --- |
275.12|

| Ave visit spend not target |  
| --- |
71.35|

- Work out the average overall spend of those who bought the target product at least once in the period vs those who did not

| Ave period spend target |  
| --- |
641|

| Ave period spend not target |  
| --- |
6.55|

In [ ]:
# read in transaction_table.csv and basket_table.csv with pandas and then use a pandas to_sql method 
# to create and populate two database tables



In [ ]:
# One way to create the visits table if you are stuck ...

# Do a select  from a subquery
# The subquery would query the basket table.
# It would sum up transaction totals, and flag whether or not a target product was purchased 
# in a transaction (hint try MAX(Case when ...)) and then group by transaction ID.
# The outer select would select customer and date from the transaction table and 
# the sum of the totals in the subquery and the max target flag from the sub query.  
# To get the date and the customer code, a join to the transactions table would be needed
# The outer select would group by transaction date and customer




In [ ]:
#Average spend per visit where the target product was purchased 


In [ ]:
#Average spend per visit where the target product was not purchased 



In [ ]:
#Average overall spend of those who bought the target product at least once in the period




In [ ]:
#Average overall spend of those who did not buy the target product at least once in the period



In [ ]:
cursor.close()
connection.close()

In [ ]:
%%javascript
require(['base/js/utils'],
function(utils) {
   utils.load_extensions('calico-spell-check', 'calico-document-tools', 'calico-cell-tools');
});